In [ ]:
f = open('1435449602.xml', 'r')
article = f.read()
f.close()

## 將XML 資料讀取進成xml 架構

In [2]:
from xml.etree import ElementTree
with open('1435449602.xml','r',encoding='utf-8') as f:
    events=ElementTree.fromstring(f.read())

type(events)

xml.etree.ElementTree.Element

## 從item 之中分別取得標題與內文，並進行切詞

In [12]:
import jieba, re

corpus = []
titles = []
for elem in events.findall('./channel/item'):
    title       = elem.find('title').text
    description = elem.find('description').text
    titles.append(title)
    article = []
    for word in jieba.cut(description):
        if re.match('[\u4e00-\u9fa5]+', word):
            article.append(word)
    corpus.append(' '.join(article))

## 建立詞頻矩陣

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [14]:
X.shape

(147, 12247)

In [16]:
word = vectorizer.get_feature_names()
#word

## 建立 tf-idf 矩陣

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf       = transformer.fit_transform(X)
tfidf.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [19]:
tfidf.shape

(147, 12247)

## 使用Kmeans 進行分群

In [21]:
from sklearn import cluster
#?cluster.KMeans
c=cluster.KMeans(n_clusters=4)

In [22]:
weight = tfidf.toarray()
k_data = c.fit_predict(weight)

## 使用Enumerate 取出分群結果

In [24]:
a= [2,5,3,4,7]
for idx, e in enumerate(a):
    print(idx, e)

0 2
1 5
2 3
3 4
4 7


In [40]:
for pos, group in enumerate(k_data):
    if group == 1:
        print(titles[pos])

蔡依林淚奪金曲 錦榮傳訊恭喜
陳奕迅、張惠妹稱王封后  蔡依林抱回最大獎
陳奕迅、莫文蔚伴侶均不知阿娜答金曲獲獎
金曲26／蔡依林擒３獎大勝　淚崩再挺婚姻平權
金曲26／張惠妹奪歌后卻失落　要世界感受彩虹力量
金曲26／蔡依林淚奪最佳專輯＋完整得獎名單
僅次Jolin！徐佳瑩入圍6獎全槓被封遺珠
金曲最風光！蔡依林紅毯全勝又獲3獎成大贏家
張惠妹3度封后  想破江蕙紀錄
金曲26／陳奕迅稱王謝台灣　張惠妹封后秒噴淚
蔡依林呸大贏家  金曲最佳專輯獎
金曲獎完整得獎名單！阿妹封后 陳奕迅稱王
第26屆金曲獎 陳奕迅奪歌王、阿妹封歌后
金曲最佳國語專輯：呸
《金曲26》2015金曲獎得獎名單 線上直播懶人包


## 使用Numpy 取出分群結果

In [39]:
import numpy
new_titles = numpy.array(titles)
new_titles[k_data == 1]

array(['蔡依林淚奪金曲 錦榮傳訊恭喜', '陳奕迅、張惠妹稱王封后  蔡依林抱回最大獎', '陳奕迅、莫文蔚伴侶均不知阿娜答金曲獲獎',
       '金曲26／蔡依林擒３獎大勝\u3000淚崩再挺婚姻平權', '金曲26／張惠妹奪歌后卻失落\u3000要世界感受彩虹力量',
       '金曲26／蔡依林淚奪最佳專輯＋完整得獎名單', '僅次Jolin！徐佳瑩入圍6獎全槓被封遺珠',
       '金曲最風光！蔡依林紅毯全勝又獲3獎成大贏家', '張惠妹3度封后  想破江蕙紀錄',
       '金曲26／陳奕迅稱王謝台灣\u3000張惠妹封后秒噴淚', '蔡依林呸大贏家  金曲最佳專輯獎',
       '金曲獎完整得獎名單！阿妹封后 陳奕迅稱王', '第26屆金曲獎 陳奕迅奪歌王、阿妹封歌后', '金曲最佳國語專輯：呸',
       '《金曲26》2015金曲獎得獎名單 線上直播懶人包'], 
      dtype='<U26')

## 取出分群結果關鍵字

In [54]:
import numpy as np
np.sum([
        [0, 1], 
        [0, 5]
       ], axis=0)

array([0, 6])

In [77]:
#group1         = tfidf[k_data == 2].toarray()
group1         = tfidf[k_data == 1].toarray()

# 計算colsums
group1_colsums = np.sum(group1, axis = 0)

# 根據colsums 做由大到小的排序
swd = group1_colsums.argsort()[::-1]

# 將每個column keyword 變成numpy array
keywords = numpy.array(vectorizer.get_feature_names())
for term, cnt in zip(keywords[swd][0:15], group1_colsums[swd][0:15]):
    print(term, cnt)

最佳 4.04617459463
專輯 2.82520550006
金曲 1.94067007063
蔡依林 1.64389056436
國語 1.58337074329
有限公司 1.2177551509
音樂 1.20907925857
張惠妹 1.1492862655
開心 1.10664627155
演唱 0.963153943651
股份 0.927744639562
專輯獎 0.921736618957
獲得 0.874320017843
歌曲 0.844433460573
典禮 0.781024150118


In [53]:
word = numpy.array(vectorizer.get_feature_names())
word[np.sum(tfidf[k_data == 1].toarray(), axis = 0).argsort()[::-1]][1:10]

array(['專輯', '金曲', '蔡依林', '國語', '有限公司', '音樂', '張惠妹', '開心', '演唱'], 
      dtype='<U7')

## 抓取各群關鍵字函式

In [90]:

def getGroupKeyword(group_id, top_k):
    group1         = tfidf[k_data == group_id].toarray()

    # 計算colsums
    group1_colsums = np.sum(group1, axis = 0)

    # 根據colsums 做由大到小的排序
    swd = group1_colsums.argsort()[::-1]

    # 將每個column keyword 變成numpy array
    keywords = numpy.array(vectorizer.get_feature_names())
    for term, cnt in zip(keywords[swd][0:top_k], group1_colsums[swd][0:top_k]):
        print(group_id, term, cnt)

In [91]:
for group_id in numpy.unique(k_data):
    getGroupKeyword(group_id, 10)

0 希臘 4.9108234637
0 歐元區 2.33005924094
0 債權 0.669524125324
0 公投 0.658834077342
0 政府 0.655118015554
0 債務 0.63652134643
0 提款 0.621601187768
0 可能 0.582126641745
0 財政部長 0.540912069506
0 櫃員機 0.529374471226
1 最佳 4.04617459463
1 專輯 2.82520550006
1 金曲 1.94067007063
1 蔡依林 1.64389056436
1 國語 1.58337074329
1 有限公司 1.2177551509
1 音樂 1.20907925857
1 張惠妹 1.1492862655
1 開心 1.10664627155
1 演唱 0.963153943651
2 樂園 4.14330231395
2 八仙 4.12942243844
2 新北市 3.93388532716
2 醫院 3.1762003018
2 現場 2.23152989783
2 爆炸 2.22054448284
2 粉塵 1.87660964446
2 彩色 1.82383967746
2 晚間 1.76313521462
2 全力 1.74433563207
3 西瓜 1.66460170799
3 台灣 1.6225689309
3 中國 1.27768694508
3 美國 1.1794085543
3 日本 1.09012430195
3 表示 1.03101524258
3 問題 1.02815489241
3 國民黨 1.01940906769
3 金曲 1.01697909732
3 改革 0.99394349865
